In [ ]:
#
"""
pandas의 분석기능
-통계처리함수 : mean, var, std(standard deviation), cov, corr(-1 ~ 1) <- ★
-데이터 요약 : groupby, cross_table (교차분석표-카운트), pivot(값무변), pivot_table(함수) <- 오전에!
    - aggregation, fillter, transform, apply
-시간과 시계열 분석(time series)의 전처리 도구
    -자기상관성을 가짐(auto corelation)
    -rnn을 이해할때 도움이 됨(시간순서를 가진 : 언어, 음성 )    
    -shift(이동), diff(차분-미분-이전값을뺌), rolling(이동평균법), resampling(데이터를 규칙화 : 샘플의 갯수를 맞춘다), pct_change(퍼센트로 잰다)
        -차분을 하는이유(변화량 정규화해서 데이터를 정상성(시간에 따른 일정한변화)을 띄게 만들기 위해)
    -window 
"""

In [ ]:
"""
범주형(이산적), 연속형 변수 172.5
 분류문제(상중하) 예측( 부동산 가격, 키의 예측)
 독립변수 연속형, 종속변수 범주형 : logistic regression(로지스틱회귀)
  ex: 소득, 부채, 식구 등등  => 대출여부(Y/N)
 독립변수 연속형, 종속변수 연속형 : 회귀분석, 상관분석
  ex: 공단에서의 거리, 고속도로의 거리, 방의 개수 .... , 종속변수는 : 주택의 가격

두 변수를 구별해서 선별하는게 중요
"""

In [12]:
#범주형인 경우
import pandas as pd
import numpy as np

# 계층적인덱스
arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
         ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]
index = pd.MultiIndex.from_arrays(arrays, names=['first', 'second'])
s = pd.Series(np.random.randn(8), index=index)
s

first  second
bar    one      -1.632564
       two       0.398206
baz    one      -1.412594
       two      -0.806220
foo    one      -1.028313
       two       2.083840
qux    one      -0.338315
       two       0.927127
dtype: float64

In [4]:
grouped = s.groupby(level = 0) # 디비명령어 차용
grouped.sum()

first
bar   -2.077981
baz   -0.017821
foo   -2.238957
qux   -2.960908
dtype: float64

In [5]:
s.groupby(level='second').sum()

second
one   -4.458040
two   -2.837626
dtype: float64

In [6]:
s.groupby(level=['first', 'second']).sum()

first  second
bar    one      -1.322567
       two      -0.755414
baz    one      -0.466745
       two       0.448924
foo    one      -0.637523
       two      -1.601434
qux    one      -2.031206
       two      -0.929702
dtype: float64

In [14]:
arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
           ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]
index = pd.MultiIndex.from_arrays(arrays, names=['first', 'second'])
df = pd.DataFrame({'A': [1, 1, 1, 1, 2, 2, 3, 3],
                   'B': np.arange(2,10)},
                  index=index)
df 
# s = pd.Series(np.random.randn(8), index=index)
# s

A  B
first second      
bar   one     1  2
      two     1  3
baz   one     1  4
      two     1  5
foo   one     2  6
      two     2  7
qux   one     3  8
      two     3  9

In [38]:
df.groupby([pd.Grouper(level=1), 'A']).sum() # 그룹으로 묶이고, 계산은 집계함수 (없으면 비출력)

B
second A   
one    1  6
       2  6
       3  8
two    1  8
       2  7
       3  9

In [18]:
index = pd.date_range('10/1/1999', periods=1100) # 주기로 ( 디폴트 주기는 일 )
print(index)
ts = pd.Series(np.random.normal(0.5, 2, 1100), index) # 1100개의 데이터
# 100개의 데이터가 하나의 묶음으로 ( 100일이 다음날에 영향을 미친다 )
ts = ts.rolling(window=100, min_periods=100).mean().dropna() # 이동평균법
ts.head()
ts.tail() 

DatetimeIndex(['1999-10-01', '1999-10-02', '1999-10-03', '1999-10-04',
               '1999-10-05', '1999-10-06', '1999-10-07', '1999-10-08',
               '1999-10-09', '1999-10-10',
               ...
               '2002-09-25', '2002-09-26', '2002-09-27', '2002-09-28',
               '2002-09-29', '2002-09-30', '2002-10-01', '2002-10-02',
               '2002-10-03', '2002-10-04'],
              dtype='datetime64[ns]', length=1100, freq='D')


2002-09-30    0.557985
2002-10-01    0.513006
2002-10-02    0.487777
2002-10-03    0.526965
2002-10-04    0.469290
Freq: D, dtype: float64

In [23]:
transformed = (ts.groupby(lambda x: x.year).transform(lambda x: (x-x.mean()) / x.std())) # std표준편차
transformed

2000-01-08   -1.027571
2000-01-09   -1.194265
2000-01-10   -1.166996
2000-01-11   -1.067052
2000-01-12   -0.766464
                ...   
2002-09-30    1.391705
2002-10-01    1.130352
2002-10-02    0.983762
2002-10-03    1.211464
2002-10-04    0.876340
Freq: D, Length: 1001, dtype: float64

In [24]:
grouped = ts.groupby(lambda x: x.year)
grouped.mean()

2000    0.639174
2001    0.616831
2002    0.318471
dtype: float64

In [25]:
grouped_trans = transformed.groupby(lambda x: x.year)
grouped_trans.mean()

2000    2.673272e-15
2001   -1.813466e-15
2002   -7.006028e-16
dtype: float64

In [28]:
sf = pd.Series([1, 1, 2, 3, 3, 3])
print(sf)
sf.groupby(sf).filter(lambda x: x.sum() > 2)

0    1
1    1
2    2
3    3
4    3
5    3
dtype: int64


3    3
4    3
5    3
dtype: int64

In [40]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df 

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [41]:
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

In [44]:
df.groupby('key').aggregate({'data1' : 'min', 'data2':'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


In [46]:
df.groupby('key').transform(lambda x: x - x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


In [49]:
def norm_by_data2(x):
    x['data1'] /= x['data2'].sum()
    return x

df.groupby("key").apply(norm_by_data2)

,key,data1,data2
0,A,0.000000,5
1,B,0.142857,0
2,C,0.166667,3
3,A,0.375000,3
4,B,0.571429,7
5,C,0.416667,9


In [15]:
df = pd.DataFrame({'A' : ['일반', '일반', '특수', '예외'] * 6,
                   'B' : ['서울', '대구', '부산'] * 8,
                   'C' : ['건조', '건조', '건조', '우기', '우기', '우기'] * 4,
                   'D' : np.random.randn(24),
                   'E' : np.random.randn(24)})
df

,A,B,C,D,E
0,일반,서울,건조,0.590598,1.013641
1,일반,대구,건조,-1.582329,0.457962
2,특수,부산,건조,1.788444,-1.823640
3,예외,서울,우기,-1.180807,-1.256753
4,일반,대구,우기,-1.160815,0.101104
5,일반,부산,우기,0.386031,-0.842772
6,특수,서울,건조,0.096349,-1.483260
7,예외,대구,건조,1.187668,-1.276959
8,일반,부산,건조,-0.779911,1.100994
9,일반,서울,우기,0.943991,2.044499


In [54]:
df.A.unique() # 범주
df.B.unique() # 범주

array(['서울', '대구', '부산'], dtype=object)

In [58]:
pd.crosstab(df.A, df.B, margins=True) # count, 교차분석표 

B,대구,부산,서울,All
A,,,,
예외,2,2,2,6
일반,4,4,4,12
특수,2,2,2,6
All,8,8,8,24


In [59]:
pd.crosstab([df.A, df.B], df.C, margins=True) # count, 교차분석표 

C       건조  우기  All
A   B              
예외  대구   2   0    2
    부산   0   2    2
    서울   0   2    2
일반  대구   2   2    4
    부산   2   2    4
    서울   2   2    4
특수  대구   0   2    2
    부산   2   0    2
    서울   2   0    2
All     12  12   24

In [60]:
pd.crosstab(df.A, df.B).apply(lambda r: r/r.sum(), axis=1)

B,대구,부산,서울
A,,,
예외,0.333333,0.333333,0.333333
일반,0.333333,0.333333,0.333333
특수,0.333333,0.333333,0.333333


In [16]:
d = {#dict
    'Name':['영구','일구','이구','삼구','사구','오구', '육구','칠구','팔구','박진','비옴','최훈'],
    'Exam':['중간고사','중간고사','중간고사','중간고사','중간고사','중간고사', '기말고사','기말고사','기말고사','기말고사','기말고사','기말고사'],
    
    'Subject':['수학','수학','수학','과학','과학','과학',  '수학','수학','수학','과학','과학','과학'],
    'Result':['Pass','Pass','Fail','Pass','Fail','Pass','Pass','Fail','Fail','Pass','Pass','Fail'],
    'Jumsu' : [ 100,80,70,60.,70,80,100,80,60,100,80,80]

} 
"""
1)과목별 패스 여부를 확인하시오
2)과목별로 분류하고 시험분별로  분류된 패스 여부를 확인하고 합계를 출력
3)시험별로 패스 여부를 확인하시오
4)시험별 이름별 점수를 확인하시오(범주형일때는 카운팅)
5)시험별 과목별 점수를 확인하시오

"""
qdf = pd.DataFrame(d)
qdf

,Name,Exam,Subject,Result,Jumsu
0,영구,중간고사,수학,Pass,100.0
1,일구,중간고사,수학,Pass,80.0
2,이구,중간고사,수학,Fail,70.0
3,삼구,중간고사,과학,Pass,60.0
4,사구,중간고사,과학,Fail,70.0
5,오구,중간고사,과학,Pass,80.0
6,육구,기말고사,수학,Pass,100.0
7,칠구,기말고사,수학,Fail,80.0
8,팔구,기말고사,수학,Fail,60.0
9,박진,기말고사,과학,Pass,100.0


In [74]:
pd.crosstab(qdf.Subject, qdf.Result, margins=True)

Result,Fail,Pass,All
Subject,,,
과학,2,4,6
수학,3,3,6
All,5,7,12


In [76]:
pd.crosstab([qdf.Subject, qdf.Exam], qdf.Result, margins=True)

Result        Fail  Pass  All
Subject Exam                 
과학      기말고사     1     2    3
        중간고사     1     2    3
수학      기말고사     2     1    3
        중간고사     1     2    3
All              5     7   12

In [17]:
pd.crosstab( qdf.Exam, qdf.Result, margins=True)

Result,Fail,Pass,All
Exam,,,
기말고사,3,3,6
중간고사,2,4,6
All,5,7,12


In [18]:
pd.crosstab(qdf.Exam, qdf.Name, qdf.Jumsu, aggfunc=[np.sum], margins=True)

sum                                                                \
Name     박진    비옴    사구    삼구     영구    오구     육구    이구    일구    최훈    칠구   
Exam                                                                        
기말고사  100.0  80.0   NaN   NaN    NaN   NaN  100.0   NaN   NaN  80.0  80.0   
중간고사    NaN   NaN  70.0  60.0  100.0  80.0    NaN  70.0  80.0   NaN   NaN   
All   100.0  80.0  70.0  60.0  100.0  80.0  100.0  70.0  80.0  80.0  80.0   

                   
Name    팔구    All  
Exam               
기말고사  60.0  500.0  
중간고사   NaN  460.0  
All   60.0  960.0

In [79]:
pd.crosstab(qdf.Exam, qdf.Subject, qdf.Jumsu, aggfunc=[np.average], margins=True)

average                      
Subject         과학         수학        All
Exam                                    
기말고사     86.666667  80.000000  83.333333
중간고사     70.000000  83.333333  76.666667
All      78.333333  81.666667  80.000000

In [81]:
df = pd.DataFrame({"foo": ['one', 'one-1', 'two', 'two'],
                    "bar": ['A', 'A', 'B', 'C'],
                    "baz": [1, 2, 3, 4]})
print(df)
df.pivot(index='foo', columns='bar', values='baz') 
# pivot 기본이 값
# crosstab은 기본이 갯수

     foo bar  baz
0    one   A    1
1  one-1   A    2
2    two   B    3
3    two   C    4


bar,A,B,C
foo,,,
one,1.0,NaN,NaN
one-1,2.0,NaN,NaN
two,NaN,3.0,4.0


In [19]:
"""
도시연도별로 인구를 파악해보시오
"""
data = {
    "도시": [  "서울", "서울", "서울", "부산", "부산", "부산", "인천", "인천"   ],
    "연도": [  "2015", "2010", "2005", "2015", "2010", "2005", "2015", "2010"    ],
    "인구": [  9904312, 9631482, 9762546, 3448737, 3393191, 3512547, 2890451, 2632035  ],
    "지역": [  "수도권", "수도권", "수도권", "경상권", "경상권", "경상권", "수도권", "수도권"    ]
} 
qdf2 = pd.DataFrame(data)
qdf2

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,2632035,수도권


In [20]:
qdf2.pivot('도시', '연도', '인구')

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,2632035.0,2890451.0


In [21]:
df = pd.read_excel("sales-funnel.xlsx")
df.head()

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


In [22]:
df["Status"] = df["Status"].astype("category") #숫자변수만
pd.pivot_table(df, index=["Name"])

,Account,Price,Quantity
Name,,,
Barton LLC,740150,35000,1.000000
"Fritsch, Russel and Anderson",737550,35000,1.000000
Herman LLC,141962,65000,2.000000
Jerde-Hilpert,412290,5000,2.000000
"Kassulke, Ondricka and Metz",307599,7000,3.000000
Keeling LLC,688981,100000,5.000000
Kiehn-Spinka,146832,65000,2.000000
Koepp Ltd,729833,35000,2.000000
Kulas Inc,218895,25000,1.500000


In [23]:
pd.pivot_table(df, index=["Name"], columns=["Manager"])

Account                      Price  \
Manager                      Debra Henley Fred Anderson Debra Henley   
Name                                                                   
Barton LLC                       740150.0           NaN      35000.0   
Fritsch, Russel and Anderson     737550.0           NaN      35000.0   
Herman LLC                            NaN      141962.0          NaN   
Jerde-Hilpert                    412290.0           NaN       5000.0   
Kassulke, Ondricka and Metz           NaN      307599.0          NaN   
Keeling LLC                           NaN      688981.0          NaN   
Kiehn-Spinka                     146832.0           NaN      65000.0   
Koepp Ltd                             NaN      729833.0          NaN   
Kulas Inc                        218895.0           NaN      25000.0   
Purdy-Kunde                           NaN      163416.0          NaN   
Stokes LLC                            NaN      239344.0          NaN   
Trantow-Barrows                  714466.0           NaN      15000.0   

                                               Quantity                
Manager                      Fred Anderson Debra Henley Fred Anderson  
Name                                                                   
Barton LLC                             NaN     1.000000           NaN  
Fritsch, Russel and Anderson           NaN     1.000000           NaN  
Herman LLC                         65000.0          NaN           2.0  
Jerde-Hilpert                          NaN     2.000000           NaN  
Kassulke, Ondricka and Metz         7000.0          NaN           3.0  
Keeling LLC                       100000.0          NaN           5.0  
Kiehn-Spinka                           NaN     2.000000           NaN  
Koepp Ltd                          35000.0          NaN           2.0  
Kulas Inc                              NaN     1.500000           NaN  
Purdy-Kunde                        30000.0          NaN           1.0  
Stokes LLC                          7500.0          NaN           1.0  
Trantow-Barrows                        NaN     1.333333           NaN

In [24]:
pd.pivot_table(df, index=["Name"], columns=["Manager"], values=["Price"])

Price              
Manager                      Debra Henley Fred Anderson
Name                                                   
Barton LLC                        35000.0           NaN
Fritsch, Russel and Anderson      35000.0           NaN
Herman LLC                            NaN       65000.0
Jerde-Hilpert                      5000.0           NaN
Kassulke, Ondricka and Metz           NaN        7000.0
Keeling LLC                           NaN      100000.0
Kiehn-Spinka                      65000.0           NaN
Koepp Ltd                             NaN       35000.0
Kulas Inc                         25000.0           NaN
Purdy-Kunde                           NaN       30000.0
Stokes LLC                            NaN        7500.0
Trantow-Barrows                   15000.0           NaN

In [100]:
pd.pivot_table(df, index=["Manager", "Rep"], values=["Price"], aggfunc=np.sum)

Price
Manager       Rep                  
Debra Henley  Craig Booker    80000
              Daniel Hilton  115000
              John Smith      40000
Fred Anderson Cedric Moss    110000
              Wendy Yule     177000

In [ ]:
# 그룹바이-DB에서가져옴, 피벗, 피벗테이블(피벗안쓰고 이것만 써도 됌)-갯수중심, 크로스테이블-값중심 네가지를 함

In [101]:
# 시험 15문제 낼거임
# 오전 종료 ===========================================================================================================================================================================================================